In [1]:
import argparse
import sys
import os
import json
import numpy as np
from osgeo import gdal, osr
from shapely.geometry import shape, mapping
from download_geojson import create_geojson
import pandas as pd
import geopandas as gpd
from shapely import wkt
import glob
import matplotlib.pyplot as plt
import json, requests, os
from datetime import datetime
import rasterio
import pyodbc
import database.Database as Database
import pytz
import database.credentialsDB as credentialsDB
from pathlib import Path
import database.credentialsDB as  accessDB
from psycopg2 import sql
import psycopg2
import extract_md as ex
from dotenv import load_dotenv
import os
import change_path_to_url as ptu

In [2]:
geojson_file_path = 's2_polska_2023_2024_wkt.geojson'

cloud_cover_range = '[0,30]'
start_date = '2023-02-01'
completion_date = '2024-12-31'

geojson = create_geojson(cloud_cover_range, start_date, completion_date)

with open(geojson_file_path, 'w') as f:
    json.dump(geojson, f)



In [3]:
# Extract the features from JSON
features = geojson['features']

# Prepare a list to store geometries and properties
geometries = []
properties = []

for feature in features:
    # Parse the geometry
    geom = shape(feature['geometry'])
    geometries.append(geom)
    
    # Get properties
    props = feature['properties']
    properties.append(props)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(properties, geometry=geometries)

In [4]:
# Extract the path_list from the 'product_identifier' column
path_list = [
    feature['properties']['product_identifier']
    for feature in geojson['features']
]
path_list = path_list[:20]
path_list

['/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/11/26/S2A_MSIL2A_20231126T100331_N0509_R122_T33UWU_20231126T142852.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/11/29/S2A_MSIL2A_20231129T101351_N0509_R022_T33UXT_20231129T140750.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/12/05/S2B_MSIL2A_20231205T094259_N0509_R036_T34UCA_20231205T113326.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/12/05/S2B_MSIL2A_20231205T094259_N0509_R036_T34UDE_20231205T113326.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/10/17/S2A_MSIL2A_20231017T100031_N0509_R122_T33UXA_20231017T142759.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/10/02/S2B_MSIL2A_20231002T095739_N0509_R122_T33UXR_20231002T130029.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/11/02/S2B_MSIL2A_20231102T093049_N0509_R136_T34UGB_20231102T111616.SAFE',
 '/eodata/Sentinel-2/MSI/L2A/2023/10/30/S2A_MSIL2A_20231030T101141_N0509_R022_T33UWS_20231030T142401.SAFE',
 '/eodata/Sentinel-2/MSI/L2A

In [5]:
#to ponizej puszczacie w termianlu i wyciagacie swoje klucze do eodaty, ktore pozniej wklejacie w miejscce
# curl http://169.254.169.254/openstack/latest/vendor_data2.json

# Wczytaj zmienne środowiskowe z pliku .env
load_dotenv()

# Teraz zmienne środowiskowe są dostępne i możesz je ustawić w kodzie
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')

# Pozostałe zmienne
os.environ['GDAL_HTTP_TCP_KEEPALIVE'] = "YES"
os.environ['AWS_S3_ENDPOINT'] = "eodata.cloudferro.com"
os.environ['AWS_HTTPS'] = "YES"
os.environ['AWS_VIRTUAL_HOSTING'] = "FALSE"

In [6]:
import subprocess
def getFullPath(var):
    # Run the s5cmd command and capture the output
    command = ['s5cmd', '--endpoint-url', 'https://eodata.cloudferro.com', 'ls',
               's3:/' + var + '/GRANULE/*/IMG_DATA/R20m/*.jp2']
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output, error = process.communicate()
    # Check if there was any error
    if process.returncode != 0:
        print(f"Error: {error.decode('utf-8')}")
        return None
    else:
        # Parse the output and extract the relative paths
        output_text = output.decode('utf-8')
        lines = output_text.split('\n')
        relative_paths = [line.split()[-1] for line in lines if line.strip()]
        return [var + '/GRANULE/' + path for path in relative_paths]

In [ ]:
for directory_path in path_list:
    print(f"Contents of {directory_path}:")
    
    matching_directories = glob.glob(os.path.join(directory_path, "GRANULE/*/IMG_DATA/R20m"))

    for directory in matching_directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                print(os.path.join(root, file))

Contents of /eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE/GRANULE/L2A_T33UXS_A043920_20231119T101314/IMG_DATA/R20m/T33UXS_20231119T101311_AOT_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE/GRANULE/L2A_T33UXS_A043920_20231119T101314/IMG_DATA/R20m/T33UXS_20231119T101311_B01_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE/GRANULE/L2A_T33UXS_A043920_20231119T101314/IMG_DATA/R20m/T33UXS_20231119T101311_B02_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE/GRANULE/L2A_T33UXS_A043920_20231119T101314/IMG_DATA/R20m/T33UXS_20231119T101311_B03_20m.jp2
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.

In [8]:
import os
import datetime
import numpy as np
import rasterio
from rasterio.io import MemoryFile
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Define a continuous colormap with 256 unique colors
spectral_colormap = plt.get_cmap('Spectral', 256)

# Generate the colormap for 256 values
ndci_colormap = {i: tuple(int(c*255) for c in spectral_colormap(i)[:3]) for i in range(256)}

def extract_date_and_tile_id(identifier):
    try:
        date_part = "/".join(identifier.split('/')[5:8])
        date_obj = datetime.datetime.strptime(date_part, '%Y/%m/%d')
        tile_id = identifier.split('_')[-2]
        return date_obj.year, date_obj.month, date_obj.day, tile_id
    except Exception as e:
        print(f"Error extracting date and tile ID from identifier {identifier}: {e}")
        return None, None, None, None

output_base_path = '../output_ndci/'
os.makedirs(output_base_path, exist_ok=True)
os.chmod(output_base_path, 0o777)

ndci_list = []

for directory_path in path_list:
    print(f"Calculating NDCI for {directory_path}:")

    matching_directories = getFullPath(directory_path)
    identifier = directory_path  # Adjust as needed
    year, month, day, tile_id = extract_date_and_tile_id(identifier)
    
    if year is None or month is None or day is None:
        print(f"Skipping directory {directory_path} due to missing date or tile information.")
        continue

    subs = ["B04_20m.jp2", "B05_20m.jp2", "SCL_20m.jp2"]
    red_band_path, red_edge_band_path, scl_path = list(filter(lambda x: any(sub in x for sub in subs), matching_directories))
    print(red_band_path)

    with rasterio.open(f's3:/{red_band_path}') as red_src, \
         rasterio.open(f's3:/{red_edge_band_path}') as red_edge_src, \
         rasterio.open(f's3:/{scl_path}') as scl_src:
        
        red_band = red_src.read(1).astype(float)
        red_edge_band = red_edge_src.read(1).astype(float)

        red_edge_resampled = np.empty_like(red_band)
        reproject(
            source=red_edge_band,
            destination=red_edge_resampled,
            src_transform=red_edge_src.transform,
            src_crs=red_edge_src.crs,
            dst_transform=red_src.transform,
            dst_crs=red_src.crs,
            resampling=Resampling.bilinear
        )

        dst_crs = CRS.from_epsg(3857)
        transform, width, height = calculate_default_transform(
            red_src.crs, dst_crs, red_src.width, red_src.height, *red_src.bounds)
        
        scl_resampled = np.empty((height, width), dtype=scl_src.read(1).dtype)
        reproject(
            source=scl_src.read(1),
            destination=scl_resampled,
            src_transform=scl_src.transform,
            src_crs=scl_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest
        )

        red_resampled = np.empty((height, width), dtype=np.float32)
        red_edge_resampled_final = np.empty((height, width), dtype=np.float32)

        reproject(
            source=red_band,
            destination=red_resampled,
            src_transform=red_src.transform,
            src_crs=red_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        reproject(
            source=red_edge_resampled,
            destination=red_edge_resampled_final,
            src_transform=red_src.transform,
            src_crs=red_src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )

        water_mask = (scl_resampled == 6)
        ndci = np.full_like(red_resampled, 255, dtype='uint8')

        ndci_water = (
            (
                red_edge_resampled_final[water_mask] - red_resampled[water_mask]
            ) / (red_edge_resampled_final[water_mask] + red_resampled[water_mask]) * 100
        )

        # Normalize NDCI values to 0-255
        ndci_min, ndci_max = ndci_water.min(), ndci_water.max()
        ndci_normalized = ((ndci_water - ndci_min) / (ndci_max - ndci_min) * 255).round().astype('uint8')
        
        ndci[water_mask] = ndci_normalized
        ndci_list.append(ndci)

        kwargs = red_src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height,
            'driver': 'GTiff',
            'dtype': 'uint8',
            'count': 1,
            'nodata': 255,
            'compress': 'zstd',
        })

        config = {
            "GDAL_NUM_THREADS": "ALL_CPUS",
            "GDAL_TIFF_INTERNAL_MASK": True,
            "GDAL_TIFF_OVR_BLOCKSIZE": "256",
            "BIGTIFF": "YES",
            "SPARSE_OK": "TRUE",
            "COMPRESS": "ZSTD",
        }

        with MemoryFile() as memfile:
            with memfile.open(**kwargs) as dst:
                dst.write(ndci, 1)
                dst.write_colormap(1, ndci_colormap)

            timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
            unique_id = f"{os.path.basename(directory_path)}_{timestamp}_NDCI"
            date_path = os.path.join(output_base_path, f"{year}/{month:02d}/{day:02d}")
            os.makedirs(date_path, exist_ok=True)

            output_filename = os.path.join(date_path, f"{unique_id}.tif")

            cog_translate(
                memfile.name,
                output_filename,
                cog_profiles.get("deflate"),
                config=config,
                in_memory=True,
                quiet=False
            )
            
            print(f"Saved COG to {output_filename}")


Calculating NDCI for /eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE/GRANULE/L2A_T33UXS_A043920_20231119T101314/IMG_DATA/R20m/T33UXS_20231119T101311_B04_20m.jp2


Reading input: /vsimem/1d5c0334-e9eb-49fe-9507-edba67717b3c/1d5c0334-e9eb-49fe-9507-edba67717b3c.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_ndci/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE_20241022172409_NDCI.tif


Saved COG to ../output_ndci/2023/11/19/S2A_MSIL2A_20231119T101311_N0509_R022_T33UXS_20231119T141601.SAFE_20241022172409_NDCI.tif
Calculating NDCI for /eodata/Sentinel-2/MSI/L2A/2023/11/26/S2A_MSIL2A_20231126T100331_N0509_R122_T33UWU_20231126T142852.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/11/26/S2A_MSIL2A_20231126T100331_N0509_R122_T33UWU_20231126T142852.SAFE/GRANULE/L2A_T33UWU_A044020_20231126T100348/IMG_DATA/R20m/T33UWU_20231126T100331_B04_20m.jp2


Reading input: /vsimem/c6243278-3d4e-4efa-8c8b-d3f8d30ebde4/c6243278-3d4e-4efa-8c8b-d3f8d30ebde4.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_ndci/2023/11/26/S2A_MSIL2A_20231126T100331_N0509_R122_T33UWU_20231126T142852.SAFE_20241022172431_NDCI.tif


Saved COG to ../output_ndci/2023/11/26/S2A_MSIL2A_20231126T100331_N0509_R122_T33UWU_20231126T142852.SAFE_20241022172431_NDCI.tif
Calculating NDCI for /eodata/Sentinel-2/MSI/L2A/2023/11/29/S2A_MSIL2A_20231129T101351_N0509_R022_T33UXT_20231129T140750.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/11/29/S2A_MSIL2A_20231129T101351_N0509_R022_T33UXT_20231129T140750.SAFE/GRANULE/L2A_T33UXT_A044063_20231129T101346/IMG_DATA/R20m/T33UXT_20231129T101351_B04_20m.jp2


...

Saved COG to ../output_ndci/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UFA_20231003T111133.SAFE_20241022172942_NDCI.tif
Calculating NDCI for /eodata/Sentinel-2/MSI/L2A/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UFD_20231003T111133.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UFD_20231003T111133.SAFE/GRANULE/L2A_T34UFD_A034339_20231003T092728/IMG_DATA/R20m/T34UFD_20231003T092729_B04_20m.jp2


Reading input: /vsimem/08f99879-8eea-42c3-9f8f-7c9624e7d03a/08f99879-8eea-42c3-9f8f-7c9624e7d03a.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_ndci/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UFD_20231003T111133.SAFE_20241022173006_NDCI.tif


Saved COG to ../output_ndci/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UFD_20231003T111133.SAFE_20241022173006_NDCI.tif
Calculating NDCI for /eodata/Sentinel-2/MSI/L2A/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UEV_20231003T111133.SAFE:
/eodata/Sentinel-2/MSI/L2A/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UEV_20231003T111133.SAFE/GRANULE/L2A_T34UEV_A034339_20231003T092728/IMG_DATA/R20m/T34UEV_20231003T092729_B04_20m.jp2


Reading input: /vsimem/3fd8252c-04d4-4332-b322-a2d86ef8d40e/3fd8252c-04d4-4332-b322-a2d86ef8d40e.tif

Adding overviews...
Updating dataset tags...
Writing output to: ../output_ndci/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UEV_20231003T111133.SAFE_20241022173021_NDCI.tif


Saved COG to ../output_ndci/2023/10/03/S2B_MSIL2A_20231003T092729_N0509_R136_T34UEV_20231003T111133.SAFE_20241022173021_NDCI.tif


In [9]:
#current_date = datetime.now()
formatted_date = datetime.datetime.now(pytz.timezone('Europe/Warsaw')).strftime("%Y-%m-%d %H:%M:%S")

# Construct the path to the credentials file relative to this file
credentialsDB_path = 'database/credentialsDB.txt'

credentialDB = accessDB.CredentialsDB(credentialsDB_path) #dla bazy danych

In [10]:
# Searching of tifs in location
import os

def find_tif_files(root_folder):
    tif_files = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.tif'):
                tif_files.append(os.path.join(dirpath, filename))
    return tif_files

# Przykładowe użycie
root_folder = '/home/eouser/Desktop/geoportall/output_ndci/'
tif_files = find_tif_files(root_folder)

# Wyświetlanie znalezionych plików .tif
for tif_file in tif_files:
    print(tif_file)


/home/eouser/Desktop/geoportall/output_ndci/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240827102737_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240827102753_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/01/14/S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.SAFE_20241020140310_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/01/14/S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.SAFE_20240827102448_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T33UYR_20230102T113531.SAFE_20241020140155_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_20240827102336_NDCI.tif
/home/eouser/Desktop/geoportall/output_ndci/2023/01/02/S2B_MSIL2A_20230102T095319_N0509_R079_T34UCA_20230102T113531.SAFE_202410201

In [11]:
#Base path to the output directory

output_base_path = '/home/eouser/Desktop/geoportall/output_ndci/'

# Initialize a set to keep track of product IDs in the file paths
product_ids_in_paths = set()
liczba_tif = 0
# Iterate through all files in the output directory
for root, dirs, files in os.walk(output_base_path):
    for file in files:
        if file.endswith(".tif"):
            liczba_tif = liczba_tif +1
            file_path = os.path.join(root, file)
            parts = file_path.split(os.sep)
            product_id = parts[-1].replace(".SAFE", ".tif")
            product_ids_in_paths.add(product_id)

print(liczba_tif)
# Initialize a set to keep track of matching product IDs
matching_product_ids = set()

# Check for corresponding records in the GeoDataFrame
for product_id in product_ids_in_paths:
    if not gdf[gdf['product_identifier'].str.contains(product_id.replace(".tif", ".SAFE"))].empty:
        matching_product_ids.add(product_id)
    else:
        print(f"No matching record found for product ID: {product_id}")

# Log the number of matches and mismatches
print(f"Total product IDs in file paths: {len(product_ids_in_paths)}")
print(f"Total matching product IDs: {len(matching_product_ids)}")
print(f"Total missing product IDs: {len(product_ids_in_paths - matching_product_ids)}")

# Optional: Log missing product IDs
for missing_product_id in product_ids_in_paths - matching_product_ids:
    print(f"Missing product ID: {missing_product_id}")

31
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T33UYR_20230301T142802.tif_20240827102600_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T34UCB_20230301T142802.tif_20240827102536_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230116T102351_N0509_R065_T33UVV_20230116T135854.tif_20241020140326_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T33UVS_20230301T142802.tif_20241020140432_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.tif_20240827102448_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230301T100021_N0509_R122_T33UWV_20230301T142802.tif_20240827102516_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230114T094341_N0509_R036_T34UGB_20230114T140700.tif_20241020140310_NDCI.tif
No matching record found for product ID: S2A_MSIL2A_20230103T101411_N0509_R022_T33UWV_2

In [13]:
import extract_md as ex
# Połączenie do bazy
conn = Database.Database(
    credentialDB.host,
    credentialDB.dbname,
    credentialDB.username,
    credentialDB.password,
    credentialDB.port
)
conn.connect()

# Ustawienie autocommit na true
conn.connection.autocommit = True

# Utworzenie obiektu cursor
cursor = conn.connection.cursor()

# Sprawdzenie, czy rozszerzenie PostGIS jest zainstalowane
try:
    cursor.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    print("PostGIS extension enabled.")
except Exception as e:
    print(f"Error enabling PostGIS extension: {e}")

# Dodanie przestrzennego indeksu, jeśli jeszcze nie istnieje
try:
    cursor.execute("""
        DO $$ 
        BEGIN
            IF NOT EXISTS (SELECT 1 FROM pg_indexes WHERE indexname = 'ndci_geometry_idx') THEN
                CREATE INDEX ndci_geometry_idx ON ndci USING GIST (geometry);
            END IF;
        END $$;
    """)
    print("Spatial index created or already exists.")
except Exception as e:
    print(f"Error creating spatial index: {e}")

# Ścieżka bazowa do katalogu wyjściowego
output_base_path = '/home/eouser/Desktop/geoportall/output_ndci/'

# Iteracja po wszystkich plikach w katalogu wyjściowym
for root, dirs, files in os.walk(output_base_path):
    for file in files:
        if file.endswith(".tif"):
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")  # Log the file being processed
            
            date_obj, product_id = ex.extract_metadata_from_path(file_path)
            
            if date_obj is None or product_id is None:
                print(f"Skipping file due to metadata extraction failure: {file_path}")
                continue  # Skip if the metadata extraction failed
            
            # Print extracted metadata for debugging
            print(f"Extracted metadata: date_obj={date_obj}, product_id={product_id}")




            # Get cloud cover and geometry from the GeoDataFrame
            try:
                tile_id = product_id.split('_')[-4]
                record = gdf[gdf['product_identifier'].str.contains(tile_id)]
                if record.empty:
                    print(f"No matching record found for product ID: {product_id}")
                    continue  # Skip if no matching record found

                cloud_cover = record['cc'].values[0]
                geometry = record['geometry'].iloc[0]  # Use .iloc[0] to get the first geometry

                print(f"Metadata extracted: product_id={product_id}, cloud_cover={cloud_cover}, geometry={geometry}")  # Log extracted metadata

                #Zmiana patha na link z gdala
                path_url = ptu.change_path_to_url(file_path)
                #print(path_url)


                # Insert metadata into the database
                try:
                    insert_query = sql.SQL(
                        "INSERT INTO ndci (product_id, date, cloud_cover, file_path, geometry) VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 3857))"
                    )
                    conn.execute_query(insert_query, (product_id, date_obj, cloud_cover, path_url, geometry.wkt), f"File {product_id} inserted successfully to {file_path}")
                except Exception as e:
                    print(f"Error inserting record into the database for file {file_path}: {e}")

            except Exception as e:
                print(f"Error extracting cloud cover and geometry for file {file_path}: {e}")

# Zamknięcie połączenia z bazą danych
conn.close()


Połączenie z bazą danych powiodło się
PostGIS extension enabled.
Spatial index created or already exists.
Processing file: /home/eouser/Desktop/geoportall_franek/output_ndci/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240827102737_NDCI.tif
Extracted metadata: date_obj=2023-02-28 00:00:00, product_id=S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240827102737_NDCI.tif
No matching record found for product ID: S2B_MSIL2A_20230228T094029_N0509_R036_T34UFE_20230228T110639.SAFE_20240827102737_NDCI.tif
Processing file: /home/eouser/Desktop/geoportall_franek/output_ndci/2023/02/28/S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240827102753_NDCI.tif
Extracted metadata: date_obj=2023-02-28 00:00:00, product_id=S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240827102753_NDCI.tif
No matching record found for product ID: S2B_MSIL2A_20230228T094029_N0509_R036_T34UEE_20230228T110639.SAFE_20240827102753_N

In [14]:
# Synchronizacja plików z katalogu output_cog z bucketem S3
cog='s3cmd sync /home/eouser/Desktop/geoportall/output_ndci/ s3://NDCI'

os.system(cog)


upload: '/home/eouser/Desktop/geoportall_franek/output_ndci/2023/01/01/S2B_MSIL2A_20230101T003709_N0509_R059_T54JVN_20230101T022604.SAFE_20241020135851_NDCI.tif' -> 's3://NDCI/2023/01/01/S2B_MSIL2A_20230101T003709_N0509_R059_T54JVN_20230101T022604.SAFE_20241020135851_NDCI.tif'  [1 of 21]
 310326 of 310326   100% in    0s     2.73 MB/s  done
upload: '/home/eouser/Desktop/geoportall_franek/output_ndci/2023/10/02/S2B_MSIL2A_20231002T095739_N0509_R122_T33UVS_20231002T130029.SAFE_20241020151851_NDCI.tif' -> 's3://NDCI/2023/10/02/S2B_MSIL2A_20231002T095739_N0509_R122_T33UVS_20231002T130029.SAFE_20241020151851_NDCI.tif'  [2 of 21]
 233676 of 233676   100% in    0s     7.50 MB/s  done
upload: '/home/eouser/Desktop/geoportall_franek/output_ndci/2023/10/02/S2B_MSIL2A_20231002T095739_N0509_R122_T33UWR_20231002T130029.SAFE_20241020151906_NDCI.tif' -> 's3://NDCI/2023/10/02/S2B_MSIL2A_20231002T095739_N0509_R122_T33UWR_20231002T130029.SAFE_20241020151906_NDCI.tif'  [3 of 21]
 339753 of 339753   100% 

0